In [ ]:
# Args
in_folder = "/content/SRN/"
out_folder = "/content/SRN/"

In [ ]:
token = ""
!git clone --recursive https://"{token}"@github.com/Kacper-M-Michalik/3dgs-priors.git

In [ ]:
import os
from huggingface_hub import login, HfApi

!cd /content/

In [ ]:
# Create venv for each prior model
!python3 -m venv /content/models/depth --without-pip
!python3 -m venv /content/models/normal --without-pip

# Have to manually install pip to correctly build venvs on colab
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!/content/models/depth/bin/python3 get-pip.py
!/content/models/normal/bin/python3 get-pip.py

# Verify venv's work
!/content/models/depth/bin/pip --version
!/content/models/normal/bin/pip --version

!ls -l /content/models/

In [ ]:
# Load requirements into venvs
!/content/models/depth/bin/pip install -r /content/3dgs-priors/geometry-priors/depth_requirements.txt
!/content/models/depth/bin/pip install -r /content/3dgs-priors/geometry-priors/normal_requirements.txt
# Add loading of normal requirements here

In [ ]:
# Get SRN cars dataset
%cd /content
!mkdir SRN
%cd /content/SRN
!mkdir srn_cars
%cd /content/SRN/srn_cars
!gdown --id 19yDsEJjx9zNpOKz9o6AaK-E8ED6taJWU -O cars.zip
!unzip cars.zip

In [ ]:
# Launch models to process dataset
!/content/models/depth/bin/python /content/3dgs-priors/geometry-priors/generate_depth.py --in_folder="{in_folder}" --out_folder="{out_folder}" --save_iter=10
#!/content/models/normal/bin/python /content/3dgs-priors/geometry-priors/generate_normal.py --in_folder="{in_folder}" --out_folder="{out_folder}" --save_iter=10

In [ ]:
# Upload generated dataset to HuggingFace
# Dataset follow ShapeNet layout, but are stored in Parquet format isntead of folders (due to performance/rate limiting reasons)

login()

out_path = os.path.join(in_folder, "*.parquet")
dataset = load_dataset("parquet", data_files=out_path)
dataset.push_to_hub("MVP-Group-Project/srn_cars_priors")